In [10]:
from transformers import RobertaTokenizer
# 简单测试过基础分词器的分词效果还可以，类似"XGT and GBL"，
# 并不会将XGT与GBL拆开。其中0表示开头2表示结尾。
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
text = "XGT and GBL"
print(tokenizer(text))

{'input_ids': [0, 1000, 18266, 8, 272, 7976, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [ ]:
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

# 加载分词器
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# 加载数据集
dataset = load_dataset('csv', data_files='dataset.csv')

# 分词函数，将文本转换为模型需要的格式
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# 使用分词函数处理数据集
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 指定模型的类别数量
num_labels = len(set(dataset['train']['label']))

# 加载预训练模型
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=3,              # 训练轮数
    per_device_train_batch_size=16,  # 每个设备的批量大小
    warmup_steps=500,                # 预热步数
    weight_decay=0.01,               # 权重衰减
    logging_dir='./logs',            # 日志目录
)

# 初始化训练器
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

# 开始训练
trainer.train()

# 保存参数到本地
trainer.save_model()